In [2]:
import pandas as pd
import numpy as np
import re
import time
from urllib import parse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
### [1] URL 생성관련 함수
def file_to_df():
    df = pd.read_excel('apple_iphone_url.xlsx')
    df['inch'] = df['inch'].astype(str)
    df['germany_inch'] = df['inch'].apply(lambda x: re.sub("\.",",",x))
    df['russia_rom'] = df['rom'].apply(lambda x: re.sub("gb","",x))
    
    return df
    
def url_maker(df):
    df['us_url'] = "https://www.apple.com/shop/buy-iphone/"+df['family_model']+"/"+df['inch']+"-inch-display-"+df['rom']+"-"+df['color']+"-unlocked"
    df['uk_url'] = "https://www.apple.com/uk/shop/buy-iphone/"+df['family_model']+"/"+df['inch']+"-inch-display-"+df['rom']+"-"+df['color']
    df['china_url'] = "https://www.apple.com.cn/shop/buy-iphone/"+df['family_model']+"/"+df['china_code']+"/A"
    
    df['japan_url'] = 0
    df['germany_url'] = 0
    df['russia_url'] = 0

    for i in range(len(df)):
        df['japan_url'][i] = "https://www.apple.com/jp/shop/buy-iphone/"+df['family_model'][i]+"/"+df['inch'][i]+\
                    parse.quote(df['japan_code_inch'][i])+"-"+df['rom'][i]+"-"+parse.quote(df['japan_code_color'][i])
        df['germany_url'][i] = "https://www.apple.com/de/shop/buy-iphone/"+df['family_model'][i]+"/"+df['germany_inch'][i]+"%22-display-"+\
                               df['rom'][i]+"-"+parse.quote(df['germany_code'][i]) 

        if df['family_model'][i] == "iphone-se":
            df['russia_url'][i] = "https://www.apple.com/ru/shop/buy-iphone/"+df['family_model'][i]+"/"+df['germany_inch'][i]+"-"+\
                                    "%D0%B4%D1%8E%D0%B9%D0%BC%D0%BE%D0%B2%D1%8B%D0%B9-%D0%B4%D0%B8%D1%81%D0%BF%D0%BB%D0%B5%D0%B9-"+\
                                   df['russia_rom'][i]+"%D0%B3%D0%B1-"+parse.quote(df['russia_code'][i])
        else: 
             df['russia_url'][i] = "https://www.apple.com/ru/shop/buy-iphone/"+df['family_model'][i]+"/"+\
                                    "%D0%B4%D0%B8%D1%81%D0%BF%D0%BB%D0%B5%D0%B9-"+\
                                    df['germany_inch'][i]+"-"+\
                                    "%D0%B4%D1%8E%D0%B9%D0%BC%D0%B0-"+\
                                   df['russia_rom'][i]+"%D0%B3%D0%B1-"+parse.quote(df['russia_code'][i])       

    df['india_url'] = "https://www.indiaistore.com/"+df['model']+"/"
    
    return df

### [2]크롤링 관련 코드
def select_no(browser):
    try:
        no_answer = browser.find_element_by_class_name('as-inlinetradeup-selector')
        no_answer.click()
    except: pass
    #print("click No")
          
def get_delivery_date(browser,post_code):
    #1. No 선택했는지 확인    
    browser.implicitly_wait(5)
    delivery_date = browser.find_element_by_class_name("as-purchaseinfo-dudeinfo-deliverymsg").text
    return delivery_date

def get_price_delivery(browser, url, country,i,post_code):
    browser.get(url)
    time.sleep(3)
    
    select_no(browser)
    delivery_date = get_delivery_date(browser,post_code)
    price = browser.find_element_by_class_name('localnav-price-content').text
    
    price_delivery_list = []
    price_delivery_list.append([price, delivery_date])
    return price_delivery_list

In [7]:
delivery_date_list = browser.find_elements_by_class_name("as-purchaseinfo-dudeinfo-deliverymsg")
delivery_date=[]

for i in range(len(delivery_date_list)):
    delivery_date.append(delivery_date_list[i])
    

AttributeError: 'list' object has no attribute 'text'

In [6]:
# <1> 기본 DataFrame을 세팅한다.
df = file_to_df()
df = url_maker(df)


url_list =[('usa','us_url','10001'),('uk','uk_url','sw1a2aa'), ('china', 'china_url',''), 
           ('japan', 'japan_url',''), ('germany','germany_url','10115'), 
           ('russia','russia_url','101000'), ('india', 'india_url')]

# <2> 나라를 선택하고 진행 - 0번째 url 받아서 우펀코드 설정
con_num = 0 #<--------------------[입력] usa:0, uk:1, china:2, japan:3, germany:4, russia:5, india:6

country = url_list[con_num][0]
res_col = url_list[con_num][0]+"_res"
col_nm = url_list[con_num][1]
post_code = url_list[con_num][2]
url = df[col_nm][0] #---> us_url, uk_url, china_url, japan_url, germany_url, russia_url, india_url 

#브라우저 열기  --> 우편코드 입력해주어야함  <<<---- 
browser = webdriver.Chrome()
browser.get(url)

In [76]:
# <3> 우편코드를 설정했다면, 이제 크롤링 GoGo!!!
df[res_col] = 0

for i in range(len(df['model'])):
    print(i, end = " ")
    url = df[col_nm][i]
    price_delivery_list = get_price_delivery(browser, url,country,i,post_code)
    df[res_col][i] = price_delivery_list
    
display(df)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 

In [78]:
df.to_excel('usa_3차_11.18.xlsx')